In [22]:
import Pkg
Pkg.activate("MMLDS_project")
using ProgressMeter
using DynamicalSystems, ReservoirComputing, Plots, Printf
using DifferentialEquations
using OrdinaryDiffEq
using NetCDF


#notebook not working without input
include("data.jl")
include("Reservoir_Computing.jl")

  Activating project at `c:\Users\tomfi\Desktop\Uni\3M_semester\Project_MMLDS\MMLDS_2\MMLDS_project\src\MMLDS_project`


cross_validate_esn (generic function with 1 method)

In [23]:
#Loading and changing El Nino Data
path_to_data = "sst.mon.mean.nc"

x = ncread(path_to_data, "sst") #sst = Sea Surface Temperature
x_reduced = x[121:170, 86:95, :] #only concerned with the important region for the El Nino happening

X_matirx = compress_data_matrix(x_reduced, 5, false)

#our esn accepts vectors as input
X_vector = reshape(X_matirx, (20,2040))
#print("Data successfully loaded.")

20×2040 Matrix{Float64}:
 28.0325  27.8742  28.1076  28.688   …  28.621   29.4195  29.5172  29.0083
 28.2426  28.2316  28.488   28.9117     28.3489  29.3835  29.6865  29.3908
 28.5179  28.6131  28.8473  29.1355     28.6434  29.723   29.7411  29.3795
 28.84    28.8382  29.0742  29.1841     29.4347  30.2484  29.8994  29.4668
 29.023   28.9616  29.1702  29.2372     29.4146  30.2277  29.8914  29.5535
 29.1572  29.1256  29.2598  29.2776  …  29.7037  30.4001  30.031   29.7284
 29.1264  29.117   29.196   29.2381     29.8099  30.4189  30.0984  29.8489
 28.9993  29.0324  29.076   29.1209     30.1048  30.5088  30.2152  29.9283
 28.7191  28.7542  28.6979  28.8395     30.32    30.6631  30.3864  30.0508
 28.4407  28.3725  28.2181  28.6071     30.32    30.5643  30.4019  30.1978
 28.7257  28.5573  28.5916  28.9932  …  27.2255  28.4254  29.3495  29.8018
 28.7958  28.7403  28.7715  29.102      26.6425  28.0568  29.141   29.8132
 28.8938  28.9374  28.9322  29.1185     27.5829  28.7421  29.3249  29.7972


In [24]:
#generating train, val and test set
train_data, val_data, test_data = train_val_test_split(X_vector, val_seconds = 5, test_seconds = 5)

#Define grid:
# Set up the hyperparameter grid-search
param_grid = []

reservoir_sizes = [512, 1024,2048]
spectral_radii = [0.8, 1.0, 1.2]
sparsities = [0.03, 0.05]
input_scales = [0.1,0.01]
ridge_values = [0.0, 1e-6, 1e-5]
# Take the Cartesian product of the possible values
for params in Iterators.product(reservoir_sizes, spectral_radii, sparsities, input_scales, ridge_values)
    push!(param_grid, ESNHyperparams(params...))
end

In [25]:
#train network
#@time esn, Wₒᵤₜ = cross_validate_esn(train_data, val_data, param_grid)
esn, Wₒᵤₜ = cross_validate_esn(train_data, val_data, param_grid)

ESNHyperparams(512, 0.8, 0.03, 0.1, 0.0)
Validation loss = 2.0e+13


Progress:   2%|█                                        |  ETA: 0:03:18

Progress:   3%|██                                       |  ETA: 0:09:20

Progress:   5%|██                                       |  ETA: 0:06:42

Progress:   6%|███                                      |  ETA: 0:09:04

...

Progress:  96%|████████████████████████████████████████ |  ETA: 0:00:21

Progress:  97%|████████████████████████████████████████ |  ETA: 0:00:16

Progress:  99%|█████████████████████████████████████████|  ETA: 0:00:05

Progress: 100%|█████████████████████████████████████████| Time: 0:09:36


(ESN{Int64, Matrix{Float64}, Default, NLADefault, Matrix{Float64}, RNN{typeof(NNlib.tanh_fast), Float64}, Matrix{Float64}, Matrix{Float64}, StandardStates, Int64, Matrix{Float64}}(2040, [28.032480239868164 27.874191284179688 … 28.628190994262695 29.02614402770996; 28.24264144897461 28.231639862060547 … 28.107284545898438 28.52972412109375; … ; 29.0902042388916 29.012521743774414 … 29.2639217376709 29.486040115356445; 28.802597045898438 28.66204071044922 … 29.333803176879883 29.558361053466797], Default(), NLADefault(), [-0.0976606297636513 0.0 … 0.0 0.0; 0.009560082865883973 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.02529092248424708; 0.0 0.0 … 0.0 -0.01727692765346392], RNN{typeof(NNlib.tanh_fast), Float64}(NNlib.tanh_fast, 1.0), [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0; 0.0; … ; 0.0; 0.0;;], StandardStates(), 0, [-0.9916573321342184 -0.9954442777266208 … -0.9942953629897753 -0.9943903886912049; 0.26175617736028584 -0.5031484428764026 … -0.359485

In [27]:
#predictions
Wₒᵤₜ

OutputLayer{StandardRidge{Float64}, LinearAlgebra.Adjoint{Float64, Matrix{Float64}}, Int64, Vector{Float64}}(StandardRidge{Float64}(1.0e-6), [6.1656429281607315 33.838559403927746 … 61.80221940194222 32.58230869890414; 1.614669431192014 33.295957808353755 … 59.11973952020926 30.837255694431732; … ; -11.291114081811028 49.718620014645765 … 83.44027810415648 -7.400190007608292; -15.673334224070741 52.66680425170549 … 115.15453717183307 -15.911374094678184], 20, [29.149429321289062, 28.65192413330078, 28.834640502929688, 29.18836212158203, 29.146839141845703, 29.25888442993164, 29.2716007232666, 29.335798263549805, 29.305763244628906, 29.151681900024414, 28.276582717895508, 27.650362014770508, 27.940204620361328, 28.86191749572754, 29.004562377929688, 28.879566192626953, 29.144243240356445, 29.23735809326172, 29.58588218688965, 29.58435821533203])